In [1]:
from time import sleep
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13104&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi={}'

d_list = []

#1~3をループ
for i in range(1,3):
  print("d_listの大きさは、",len(d_list))
  
  #アクセスするためのURLをtarget_urlへ入れる
  target_url = url.format(i)
  
  print(target_url)
  
  #URLにアクセスした結果を変数rに入れる
  r = requests.get(target_url)
  
  sleep(1)
  
  # BeautifulSoupでHTMLを解析
  soup = BeautifulSoup(r.text)
  
  #cassetteitemクラスをもったdivタグを全て取得して、変数contentsに格納
  contents = soup.find_all('div',class_="cassetteitem")
  
  #各物件情報をfor ループで取得する
  for content in contents:
    
    #物件・建物情報を変数detailに格納する
    detail = content.find('div',class_="cassetteitem-detail")
    
    #各部屋の情報を変数tableに格納する
    table = content.find('table',class_="cassetteitem_other")
    
    #変数titleに物件名を格納する
    title = detail.find('div',class_='cassetteitem_content-title').text
    
    #変数addressに住所を格納する
    address = detail.find('li',class_='cassetteitem_detail-col1').text
    
    #変数accessにアクセス情報を格納する
    access = detail.find('li',class_='cassetteitem_detail-col2').text
    
    #変数ageに築年数を格納する
    age = detail.find('li',class_='cassetteitem_detail-col3').text
    
    #変数tableから全てのtrタグを取得して、変数tr_tagsに格納
    tr_tags = table.find_all('tr',class_='js-cassette_link')
    
    #各部屋情報をforループで取得する
    for tr_tag in tr_tags:
      
      #変数floorsに階数を格納する
      floor,price,first_fee,capa = tr_tag.find_all('td')[2:6]
      
      #さらに細かい情報を取得する
      #賃料と管理費を格納
      fee,management_fee = price.find_all('li')
      
      #敷金、礼金を格納
      deposit,gratuity = first_fee.find_all('li')
      
      #間取りと面積を格納
      madori,menseki = capa.find_all('li')
      
      #辞書でこれまでに取得したデータを格納
      d = {
        'title':title,
        'address':address,
        'age':age,
        'floor':floor.text,
        'fee':fee.text,
        'management_fee':management_fee.text,
        'deposit':deposit.text,
        'gratuity':gratuity.text,
        'madori':madori.text,
        'menseki':menseki.text
      }
      
      #取得した辞書をd_listに格納する
      d_list.append(d)

d_listの大きさは、 0
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13104&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1
d_listの大きさは、 187
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13104&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=2


In [4]:
#d_listの2番目まで中身を確認
d_list[:2]

[{'title': 'TomihisaCrossComfort-富久クロスコンフォートタワー',
  'address': '東京都新宿区富久町',
  'age': '\n築8年\n地下2地上55階建\n',
  'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t4階',
  'fee': '23万円',
  'management_fee': '10000円',
  'deposit': '46万円',
  'gratuity': '23万円',
  'madori': '1LDK',
  'menseki': '47.06m2'},
 {'title': 'TomihisaCrossComfort-富久クロスコンフォートタワー',
  'address': '東京都新宿区富久町',
  'age': '\n築8年\n地下2地上55階建\n',
  'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t2階',
  'fee': '22.5万円',
  'management_fee': '15000円',
  'deposit': '22.5万円',
  'gratuity': '22.5万円',
  'madori': '1SLDK',
  'menseki': '45.7m2'}]

In [12]:
import re

In [6]:
#変数d_listを使って、データフレームを作成する
df = pd.DataFrame(d_list)

In [7]:
#データフレームの先頭５行を確認
df.head()

,title,address,age,floor,fee,management_fee,deposit,gratuity,madori,menseki
0,TomihisaCrossComfort-富久クロスコンフォートタワー,東京都新宿区富久町,\n築8年\n地下2地上55階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t4階,23万円,10000円,46万円,23万円,1LDK,47.06m2
1,TomihisaCrossComfort-富久クロスコンフォートタワー,東京都新宿区富久町,\n築8年\n地下2地上55階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,22.5万円,15000円,22.5万円,22.5万円,1SLDK,45.7m2
2,TomihisaCrossComfort-富久クロスコンフォートタワー,東京都新宿区富久町,\n築8年\n地下2地上55階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t40階,23万円,10000円,46万円,46万円,1LDK,36.22m2
3,TomihisaCrossComfort-富久クロスコンフォートタワー,東京都新宿区富久町,\n築8年\n地下2地上55階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,22.9万円,15000円,22.9万円,22.9万円,1LDK,45.7m2
4,TomihisaCrossComfort-富久クロスコンフォートタワー,東京都新宿区富久町,\n築8年\n地下2地上55階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t10階,34万円,-,34万円,34万円,2LDK,60.87m2


In [8]:
#dfの大きさを確認
df.shape

(374, 10)

In [10]:
#物件名の重複を削除して、その大きさを確認
len(df.title.unique())

30

In [13]:
df2 = df.applymap(lambda x: re.sub('\n', ' ', x))
df2.head()

,title,address,age,floor,fee,management_fee,deposit,gratuity,madori,menseki
0,TomihisaCrossComfort-富久クロスコンフォートタワー,東京都新宿区富久町,築8年 地下2地上55階建,\r \t\t\t\t\t\t\t\t\t\t\t4階,23万円,10000円,46万円,23万円,1LDK,47.06m2
1,TomihisaCrossComfort-富久クロスコンフォートタワー,東京都新宿区富久町,築8年 地下2地上55階建,\r \t\t\t\t\t\t\t\t\t\t\t2階,22.5万円,15000円,22.5万円,22.5万円,1SLDK,45.7m2
2,TomihisaCrossComfort-富久クロスコンフォートタワー,東京都新宿区富久町,築8年 地下2地上55階建,\r \t\t\t\t\t\t\t\t\t\t\t40階,23万円,10000円,46万円,46万円,1LDK,36.22m2
3,TomihisaCrossComfort-富久クロスコンフォートタワー,東京都新宿区富久町,築8年 地下2地上55階建,\r \t\t\t\t\t\t\t\t\t\t\t2階,22.9万円,15000円,22.9万円,22.9万円,1LDK,45.7m2
4,TomihisaCrossComfort-富久クロスコンフォートタワー,東京都新宿区富久町,築8年 地下2地上55階建,\r \t\t\t\t\t\t\t\t\t\t\t10階,34万円,-,34万円,34万円,2LDK,60.87m2


In [15]:
df3 = df2.applymap(lambda x: re.sub('\r', ' ', x))
df4 = df3.applymap(lambda x: re.sub('\t', ' ', x))

In [16]:
#to_csvでデータフレームをCSV出力
df4.to_csv('test2.csv',index=None,encoding='utf-8-sig')